# Stress analysis for each element 

In [50]:
name = 'yannis'

# Import modules 

In [51]:
import pandas as pd
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.panels import *
from formulas.helpers import *
from formulas.abd_matrix import *
from formulas.Strength import * 

# Import needed files 

## Import panel stresses

In [52]:
PlyStressesX = pd.read_csv(f'../data/{name}/compositePanelXX.csv')
PlyStressesX = PlyStressesX.drop(columns=['FileID', 'Step'])
PlyStressesY = pd.read_csv(f'../data/{name}/compositePanelYY.csv')
PlyStressesY = PlyStressesY.drop(columns=['FileID', 'Step'])
PlyStressesXY = pd.read_csv(f'../data/{name}/compositePanelXY.csv')
PlyStressesXY = PlyStressesXY.drop(columns=['FileID', 'Step'])
PlyStressesX

,Elements,Loadcase,Layer,Composite Stresses:Normal X Stress
0,1,1,Ply 1,-30.129749
1,2,1,Ply 1,33.933514
2,3,1,Ply 1,5.449137
3,4,1,Ply 1,-13.714887
4,5,1,Ply 1,25.343819
...,...,...,...,...
1435,26,3,Ply 9,19.149086
1436,27,3,Ply 9,-22.105305
1437,28,3,Ply 9,64.143097
1438,29,3,Ply 9,17.373724


In [53]:
PlyStressesPanel=pd.merge(PlyStressesX,PlyStressesY, on=['Elements', 'Layer', 'Loadcase'])
PlyStressesPanel = pd.merge(PlyStressesPanel, PlyStressesXY, on=['Elements', 'Layer', 'Loadcase'])
PlyStressesPanel = PlyStressesPanel.rename(columns={'Composite Stresses:Normal X Stress':'Normal_1', 'Composite Stresses:Normal Y Stress':'Normal_2', 'Composite Stresses:Shear XY Stress':'Shear_12' })
PlyStressesPanel

,Elements,Loadcase,Layer,Normal_1,Normal_2,Shear_12
0,1,1,Ply 1,-30.129749,-10.697798,11.398685
1,2,1,Ply 1,33.933514,-4.099037,4.942441
2,3,1,Ply 1,5.449137,-11.857535,12.995567
3,4,1,Ply 1,-13.714887,-10.444026,8.897148
4,5,1,Ply 1,25.343819,-5.437790,4.564201
...,...,...,...,...,...,...
1435,26,3,Ply 9,19.149086,-2.126457,-14.955009
1436,27,3,Ply 9,-22.105305,-2.504914,-15.524483
1437,28,3,Ply 9,64.143097,-1.380899,-15.699356
1438,29,3,Ply 9,17.373724,-2.618130,-12.505588


## Import stringer plies 

In [54]:
StringerStrains = pd.read_csv(f'../data/{name}/stringer_strain.csv')
StringerStrains = StringerStrains.drop(columns=['FileID', 'Step'])
StringerStrains = StringerStrains.rename(columns={'Element Strains (1D):CBAR/CBEAM Axial Strain':'strainX'})
StringerStrains

,Elements,Loadcase,strainX
0,40,1,-0.001241
1,41,1,-0.001524
2,42,1,-0.001251
3,46,1,-0.001285
4,47,1,-0.001590
5,48,1,-0.001280
6,52,1,-0.001293
7,53,1,-0.001595
8,54,1,-0.001277
9,58,1,-0.001251


# Constants for analysis 

In [55]:
# r = perpendicular 
# p = parallel
# c = compression 
# t = tension 
# Resistance values 

R_p_t = 3050
R_p_c = 1500
R_r_c = 50
R_r_t = 300 
R_rp = 100

# p values 
# Perpendicular, parallel
p_rp_c = 0.25
p_rp_t = 0.25

# Perpendicular, perpendicular
p_rr_c = 0.25
p_rr_t = 0.25

# Calculate RF and mode for panels

In [56]:
PlyStressesPanel[['Mode', 'RF_IFF', 'RF_FF']] = PlyStressesPanel.apply(strength, R_p_t=R_p_t, R_p_c=R_p_c, R_r_c=R_r_c, R_r_t=R_r_t, 
                                                    R_rp=R_rp, p_rp_c=p_rp_c, p_rp_t=p_rp_t, p_rr_c=p_rr_c, p_rr_t=p_rr_t,
                                                    axis=1, result_type='expand')
PlyStressesPanel

,Elements,Loadcase,Layer,Normal_1,Normal_2,Shear_12,Mode,RF_IFF,RF_FF
0,1,1,Ply 1,-30.129749,-10.697798,11.398685,ModeC,4.470825,49.784682
1,2,1,Ply 1,33.933514,-4.099037,4.942441,ModeC,11.527610,89.881644
2,3,1,Ply 1,5.449137,-11.857535,12.995567,ModeC,4.023417,559.721637
3,4,1,Ply 1,-13.714887,-10.444026,8.897148,ModeC,4.652374,109.370207
4,5,1,Ply 1,25.343819,-5.437790,4.564201,ModeC,8.942898,120.344927
...,...,...,...,...,...,...,...,...,...
1435,26,3,Ply 9,19.149086,-2.126457,-14.955009,ModeB,6.928643,159.276532
1436,27,3,Ply 9,-22.105305,-2.504914,-15.524483,ModeB,6.706512,67.857015
1437,28,3,Ply 9,64.143097,-1.380899,-15.699356,ModeB,6.511296,47.549934
1438,29,3,Ply 9,17.373724,-2.618130,-12.505588,ModeC,9.985093,175.552461


## Convert to output for panels 

In [ ]:
# We can drop the stresses 
PlyStressesPanel = PlyStressesPanel.drop(columns=['Normal_1', 'Normal_2', 'Shear_12'])

#Reorder plies 
PlyStressesPanel['Layer_num'] = PlyStressesPanel['Layer'].str.extract('(\d+)').astype(int)
PlyStressesPanel = PlyStressesPanel.sort_values(by='Layer_num')
PlyStressesPanel = PlyStressesPanel.drop(columns=['Layer'])
PlyStressesPanel = PlyStressesPanel.rename(columns={'Layer_num':'Layer'})

# Resort columns to desired order
PlyStressesPanel = PlyStressesPanel[['Elements', 'Layer', 'Loadcase', 'RF_FF', 'RF_IFF', 'Mode']]

# We need to analyse element 8 
PlyStressesPanel = PlyStressesPanel[PlyStressesPanel['Elements'] == 8]
#Now we separate the loadcases 
PlyStressesPanelLC1 = PlyStressesPanel[PlyStressesPanel['Loadcase'] == 1]
PlyStressesPanelLC2 = PlyStressesPanel[PlyStressesPanel['Loadcase'] == 2]
PlyStressesPanelLC3 = PlyStressesPanel[PlyStressesPanel['Loadcase'] == 3]
PlyStressesPanelLC1

<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Yanni\AppData\Local\Temp\ipykernel_17728\2277276299.py:6: SyntaxWarning: invalid escape sequence '\d'
  PlyStressesPanel['Layer_num'] = PlyStressesPanel['Layer'].str.extract('(\d+)').astype(int)


,Elements,Loadcase,RF_FF,RF_IFF,Mode,Layer
7,8,1,102.592367,9.241452,ModeC,1
247,8,1,103.547772,9.125119,ModeC,2
277,8,1,19.041854,18.155291,ModeA,3
307,8,1,18.864378,17.842834,ModeA,4
337,8,1,15.355873,21.888509,ModeA,5
367,8,1,15.068526,21.776240,ModeA,6
397,8,1,63.870059,7.427025,ModeC,7
427,8,1,62.756509,7.295658,ModeC,8
457,8,1,61.681122,7.168814,ModeC,9
37,8,1,60.641969,7.046265,ModeC,10


# Calculate stringer strength 